In [1]:
from sklearn import *
import lightgbm as lgbm
import xgboost as xgb
import catboost as cat
from sklearn.model_selection import train_test_split
from sqlalchemy import column
import pandas as pd
import numpy as np
#!pip install optuna 
import optuna

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import BayesianRidge

from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
def convert_to_category(X):
    for col in X.columns:
        if ( (X[col].dtype != 'int64') and (X[col].dtype != 'float64') and (X[col].dtype != 'bool')):
            X[col] = X[col].astype('category')
            
    return X

def remove_outliers(df, column_names, n):
    for column_name in column_names:
        mean = df[column_name].mean()
        std = df[column_name].std()
        df = df[(df[column_name] > mean - n*std) & (df[column_name] < mean + n*std)]
    return df
    
def log_func(y, shift_var):
    return np.log(y+shift_var)

def exp_func(y, shift_var):
    return np.exp(y)-shift_var

In [3]:
import numpy as np 

def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        `sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5`
    """
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))

In [13]:
stores_train = pd.read_csv('data/stores_train_with_extra_features.csv')
stores_test = pd.read_csv('data/stores_test_with_extra_features.csv')

plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')
grunnkrets = pd.read_csv('data/grunnkrets_norway_stripped.csv')
grunnkrets_ages = pd.read_csv('data/grunnkrets_age_distribution.csv')
grunnkrets_household_types = pd.read_csv('data/grunnkrets_households_num_persons.csv')
grunnkrets_household_income = pd.read_csv('data/grunnkrets_income_households.csv')

stores_train_copy= stores_train.copy()

plaace_hierarchy_copy = plaace_hierarchy.copy()
plaace_hierarchy_copy.drop(columns='sales_channel_name', inplace=True)
plaace_hierarchy_copy['lv1'] = plaace_hierarchy_copy['lv1'].astype('category')
plaace_hierarchy_copy['lv2'] = plaace_hierarchy_copy['lv2'].astype('category')


grunnkrets_copy = grunnkrets.copy()
grunnkrets_copy.rename(columns={'year': 'year_1'}, inplace=True)

grunnkrets_ages_copy = grunnkrets_ages.copy()
grunnkrets_ages_copy.rename(columns={'year': 'year_2'}, inplace=True)
grunnkrets_ages_copy['grunnkrets_population'] = grunnkrets_ages_copy.iloc[:, 2:].sum(axis=1)

grunnkrets_household_types_copy = grunnkrets_household_types.copy()
grunnkrets_household_types_copy.rename(columns={'year': 'year_3'}, inplace=True)
grunnkrets_household_types_copy[grunnkrets_household_types_copy.columns[2:]] = grunnkrets_household_types_copy[grunnkrets_household_types_copy.columns[2:]].astype('int64')

grunnkrets_household_income_copy = grunnkrets_household_income.copy()
grunnkrets_household_income_copy.rename(columns={'year': 'year_4', 'singles': 'singles_income','couple_without_children':'couple_without_children_income'}, inplace=True)
grunnkrets_household_income_copy.rename(columns={'singles': 'singles_income', }, inplace=True)

#set the values that are 0 to the lowest value in the column
# for column in grunnkrets_household_income_copy.columns[2:]:
#     grunnkrets_household_income_copy[column] = grunnkrets_household_income_copy[column].apply(lambda x: grunnkrets_household_income_copy[column].min() if x == 0 else x)
    

df = stores_train_copy

df = pd.merge(df, plaace_hierarchy_copy, on='plaace_hierarchy_id', how='left')
df = pd.merge(df, grunnkrets_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_ages_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_types_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_income_copy, on='grunnkrets_id', how='left')



df['grunnkrets_population_density'] = df['grunnkrets_population'] / df['area_km2']



df.drop_duplicates(subset=['store_id'], keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)



columns_to_drop = ['revenue',
                  'store_id',
                  'plaace_hierarchy_id',
                  'grunnkrets_id',
                  'year',
                   'address',
                  'store_name',
                  
                  'year_1',
                  'geometry',
                  
                  #'area_km2',
               
                  'grunnkrets_name',
                  'district_name',
             
                  'municipality_name',
                  
                  
                  'year_2',
                  'year_3',
                  'year_4',
                 
                
                  'sales_channel_name',
                   #'mall_name',
                  #'chain_name',
                  
                  
                  
                     'lv1',
                  'lv2',
                  'lv3',
                  'lv4',
                  
                  # 'lv1_desc',
                  # 'lv2_desc',
                  # 'lv3_desc',
                  # 'lv4_desc'
                
#                 'closest_busstop',
                  
#     'num_closest_busstops_250m',
#    'num_closest_busstops_500m',

#        'num_closest_busstops_1000m', 'num_closest_busstops_2500m',
#        'num_closest_busstops_5000m', 
#        'num_closest_busstops',
#        'num_closest_busstops_10000m', 'num_closest_busstops_15000m',
       
#        'closest_store_lv1', 'closest_store_lv2',
#        'closest_store_lv3', 'closest_store_lv4', 
       
#        'num_closest_stores_lv1_250m',   
#    'num_closest_stores_lv1_500m', 
   
#    'num_closest_stores_lv1_1000m',
    
#     'num_closest_stores_lv1_2500m',
#        'num_closest_stores_lv1_5000m', 
#        'num_closest_stores_lv1',
#        'num_closest_stores_lv1_10000m', 
# 'num_closest_stores_lv1_15000m',
       
#        'num_closest_stores_lv2_250m', 
   
   
#    'num_closest_stores_lv2_500m',
   
#     'num_closest_stores_lv2_1000m', 
    
#     'num_closest_stores_lv2_2500m',
#        'num_closest_stores_lv2_5000m',
#        'num_closest_stores_lv2',
#        'num_closest_stores_lv2_10000m', 'num_closest_stores_lv2_15000m',
       
#         'num_closest_stores_lv3_250m', 
   
   
#    'num_closest_stores_lv3_500m',
   
#        'num_closest_stores_lv3_1000m', 'num_closest_stores_lv3_2500m',
#        'num_closest_stores_lv3_5000m', 
#        'num_closest_stores_lv3',
#        'num_closest_stores_lv3_10000m', 'num_closest_stores_lv3_15000m',
       
#         'num_closest_stores_lv4_250m', 
   
#    'num_closest_stores_lv4_500m',
   
#        'num_closest_stores_lv4_1000m', 'num_closest_stores_lv4_2500m',
#        'num_closest_stores_lv4_5000m', 
#        'num_closest_stores_lv4',
#        'num_closest_stores_lv4_10000m', 'num_closest_stores_lv4_15000m',
#                   'distance_to_oslo',
                  'distance_to_bergen',
                  'distance_to_trondheim',
                  'distance_to_stavanger',
                  'distance_to_drammen',
                  # 'grunnkrets_population',
                  # 'grunnkrets_population_density',
                  
                  # 'num_closest_busstops_100m',
                  # 'num_closest_busstops_750m',
                  # 'num_closest_stores_lv1_100m',
                  # 'num_closest_stores_lv1_750m', 
                  # 'num_closest_stores_lv2_100m',
                  # 'num_closest_stores_lv2_750m',
                  # 'num_closest_stores_lv3_100m',
                  # 'num_closest_stores_lv3_750m',
              
                  # 'num_closest_stores_lv4_750m',
                  
                  ]


# fill in missing values of float columns with mean


X = df.drop(columns=columns_to_drop)
X = convert_to_category(X)

y = df.revenue
y = np.log1p(y)




categorical_features = X.select_dtypes(include=['category']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns


# One hot encoding
        
full_pipeline = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)], remainder='passthrough')
one_hot_encoder = full_pipeline.fit(X)
X_encoded_one_hot = one_hot_encoder.transform(X)

### LGBM

In [20]:
from optuna.integration import LightGBMPruningCallback
from sklearn.model_selection import KFold

random_state = 12

def objective(trial,data=X,target=y):

    params = {
        'metric': 'rmse', 
        'random_state': random_state,
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 50.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 50.0),
        #'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.1, 1),
        #'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'subsample': trial.suggest_loguniform('subsample', 0.1, 1),
        #'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02, 0.03, 0.04, 0.05]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1),
        #'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'max_depth': trial.suggest_int('max_depth', 10, 100),
        
        'num_leaves' : trial.suggest_int('num_leaves', 10, 1000),
       
        
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        
    
        
    }
    
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    kf_scores = np.empty(5)
    
    model = lgbm.LGBMRegressor(**params)
    
    
    for idx, (train_index, test_index) in enumerate(kf.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
    
        model.fit(X_train, y_train)
        
        preds = model.predict(X_test)
        
        y_test_exp = np.expm1(y_test)
        preds_exp = np.expm1(preds)
        
        preds_exp = np.where(preds_exp < 0, 0, preds_exp)
        
        kf_scores[idx] = rmsle(y_test_exp, preds_exp)

    return np.mean(kf_scores)

In [21]:
import joblib


study = optuna.create_study(direction='minimize')
joblib.dump(study, "study.pkl")

study.optimize(objective, n_trials=1000)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-11-10 07:14:28,409] A new study created in memory with name: no-name-ecce6c35-3b4b-4dd4-9abd-88fe5463eb1a
[I 2022-11-10 07:14:35,574] Trial 0 finished with value: 0.7467656161397648 and parameters: {'n_estimators': 572, 'reg_alpha': 0.0826772122656365, 'reg_lambda': 0.01629877862961394, 'colsample_bytree': 0.4928197201895361, 'subsample': 0.8654632710889733, 'learning_rate': 0.038716130988377645, 'max_depth': 50, 'num_leaves': 323, 'min_child_samples': 192}. Best is trial 0 with value: 0.7467656161397648.
[I 2022-11-10 07:14:44,397] Trial 1 finished with value: 0.7358014641968597 and parameters: {'n_estimators': 915, 'reg_alpha': 20.183172397344755, 'reg_lambda': 0.8824771276962246, 'colsample_bytree': 0.3049667454118154, 'subsample': 0.14633432228928836, 'learning_rate': 0.013197099019571747, 'max_depth': 13, 'num_leaves': 133, 'min_child_samples': 56}. Best is trial 1 with value: 0.7358014641968597.
[I 2022-11-10 07:14:47,075] Trial 2 finished with value: 0.7398430624338431 a

KeyboardInterrupt: 

In [22]:
params = study.best_trial.params
val = study.best_trial.value

params

{'n_estimators': 366,
 'reg_alpha': 0.046433293427957184,
 'reg_lambda': 7.3647658153160185,
 'colsample_bytree': 0.6504777271414033,
 'subsample': 0.11125558206697211,
 'learning_rate': 0.015734794109992907,
 'max_depth': 31,
 'num_leaves': 29,
 'min_child_samples': 197}

In [66]:
study = joblib.load("study.pkl")
joblib.dump(study, "study.pkl")
study.optimize(objective, n_trials=500)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[1]	valid_0's rmse: 0.91205
[2]	valid_0's rmse: 0.87142
[3]	valid_0's rmse: 0.830799
[4]	valid_0's rmse: 0.810656
[5]	valid_0's rmse: 0.786282
[6]	valid_0's rmse: 0.774516
[7]	valid_0's rmse: 0.775421
[8]	valid_0's rmse: 0.771357
[9]	valid_0's rmse: 0.765918
[10]	valid_0's rmse: 0.765049
[11]	valid_0's rmse: 0.762088
[12]	valid_0's rmse: 0.76228
[13]	valid_0's rmse: 0.762348
[14]	valid_0's rmse: 0.761517
[15]	valid_0's rmse: 0.761784
[16]	valid_0's rmse: 0.762861
[17]	valid_0's rmse: 0.76238
[18]	valid_0's rmse: 0.762168
[19]	valid_0's rmse: 0.761835
[20]	valid_0's rmse: 0.763012
[21]	valid_0's rmse: 0.762901
[22]	valid_0's rmse: 0.763587
[23]	valid_0's rmse: 0.764673
[24]	valid_0's rmse: 0.765118
[25]	valid_0's rmse: 0.765733
[26]	valid_0's rmse: 0.765628
[27]	valid_0's rmse: 0.767134
[28]	valid_0's rmse: 0.768213
[29]	valid_0's rmse: 0.76731
[30]	valid_0's rmse: 0.767447
[31]	valid_0's rmse: 0.767492
[32]	valid_0's rmse: 0.767899
[33]	valid_0's rmse: 0.769339
[34]	valid_0's rmse: 0.7

KeyboardInterrupt: 

In [39]:
params = study.best_trial.params
val = study.best_trial.value

params

{'n_estimators': 4579,
 'reg_alpha': 0.10988270727649117,
 'reg_lambda': 13.588639018170038,
 'colsample_bytree': 0.7794731560271715,
 'subsample': 0.3565601014644079,
 'learning_rate': 0.0018699004558419715,
 'max_depth': 10,
 'num_leaves': 294,
 'min_child_samples': 188}

In [12]:
import json
params = study.best_trial.params
val = study.best_trial.value

dict = {'val': val, 'params': params}

# save the params and value

with open('lgbm_params.json', 'w') as fp:
    json.dump(dict, fp)